In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.lines as mlines

from itertools import cycle
from matplotlib import pyplot as plt

from simulation import SimulationMixin


In [2]:
fontsize = 20

plots_dir = "plots"

In [3]:
colors = [("red", "red"), ("green", "green"), ("blue", "blue"), ("orange", "orange")]
color_cycle = cycle(colors)

In [4]:
program = "graph_coloring"
# program = "dijkstra_token_ring"
# program = "maximal_matching"
# graph_name = "graph_powerlaw_cluster_graph_n7"
# graph_name = "implicit_graph_n10"
graph_name = "graph_10"
sched = 0
no_simulations = 100000
me = False
fault_interval = 2

# duong_mode = True
simulation_type = "controlled_at_node_amit_v2"
limits = 100

selected_nodes = [9]

In [5]:
def get_filename(graph_name, sched, simulation_type, args, no_simulations, me, fault_interval):
    return f"{graph_name}__{sched}__{simulation_type}_args_{args}__{no_simulations}__{me}__{fault_interval}__limits_{limits}"

In [6]:
# filenames = [
#     get_filename(
#         graph_name,
#         sched,
#         SimulationMixin.RANDOM_FAULT_SIMULATION_TYPE,
#         "",
#         no_simulations,
#         me,
#         fault_interval,
#     )
# ]
filenames = []

filenames.extend(
    [
        get_filename(
            graph_name,
            sched,
            simulation_type,
            arg,
            no_simulations,
            me,
            fault_interval,
        )
        for arg in selected_nodes
    ]
)


In [7]:
dfs = [pd.read_csv(os.path.join("results", program, f"{fn}.csv")) for fn in filenames]

In [8]:
max_steps = max(df['Steps'].max() for df in dfs)
max_steps

np.int64(22)

In [9]:
bins = np.linspace(0, max_steps, max_steps+1)
# bin_centers = (bins[:-1] + bins[1:]) / 2
bin_centers = bins

In [10]:
bins

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
       13., 14., 15., 16., 17., 18., 19., 20., 21., 22.])

In [11]:
hist_data = [np.histogram(df['Steps'], bins=bins) for df in dfs]
hist_data[0][0]

array([    0, 41413,  2406, 25493,  3353, 13761,  2055,  5879,  1184,
        2336,   539,   828,   225,   293,    84,    77,    32,    22,
           8,     3,     7,     2])

In [12]:
hist_data = [np.histogram(df['Steps'], bins=bins) for df in dfs]
hist_df = []
for hd in hist_data:
    hist_df.append(
        pd.DataFrame({
            'Steps': bin_centers[:-1],
            'Count': hd[0]
        })
    )

In [13]:
hist_df[0]

,Steps,Count
0,0.0,0
1,1.0,41413
2,2.0,2406
3,3.0,25493
4,4.0,3353
5,5.0,13761
6,6.0,2055
7,7.0,5879
8,8.0,1184
9,9.0,2336


In [14]:
df_merged = hist_df[0]
for i in range(1, len(hist_df)):
    df_merged = pd.merge(df_merged, hist_df[i], on=["Steps"], suffixes=(i-1, i))
df_merged

,Steps,Count
0,0.0,0
1,1.0,41413
2,2.0,2406
3,3.0,25493
4,4.0,3353
5,5.0,13761
6,6.0,2055
7,7.0,5879
8,8.0,1184
9,9.0,2336


In [15]:
df_merged.set_index('Steps', drop=True, inplace=True)
df_merged

,Count
Steps,
0.0,0
1.0,41413
2.0,2406
3.0,25493
4.0,3353
5.0,13761
6.0,2055
7.0,5879
8.0,1184


In [16]:
def get_title():
    return f"Simulation - {program} | {graph_name} | Sched: {sched} | N: {no_simulations:,} | FI: {fault_interval}"

In [17]:
def get_label(simulation_type, node):
    return {
        SimulationMixin.RANDOM_FAULT_SIMULATION_TYPE: "Random Fault",
        SimulationMixin.CONTROLLED_FAULT_AT_NODE_SIMULATION_TYPE: "Controlled (amit v1) at node %s",
        SimulationMixin.CONTROLLED_FAULT_AT_NODE_SIMULATION_TYPE_AMIT_V2: "Controlled (amit v2) at node %s",
        SimulationMixin.CONTROLLED_FAULT_AT_NODE_SIMULATION_TYPE_DUONG: "Controlled (duong) at node %s",
        SimulationMixin.RANDOM_FAULT_START_AT_NODE_SIMULATION_TYPE: "Random started at node %s",
    }.get(simulation_type, simulation_type) % (node)

In [18]:
def get_filename():
    return f"{program}__{graph_name}__{sched}__{no_simulations}__{fault_interval}__{''.join([str(i) for i in selected_nodes])}__{simulation_type}__{limits}"

In [19]:
from scipy.optimize import curve_fit


def exp_func(x, a, b, c):
    return a * np.exp(-b * x) + c


# Fit the curve
popt, _ = curve_fit(exp_func, df_merged.index, df_merged["Count"], p0= [10, 0.1, 0])
y_fit = exp_func(df_merged.index, popt[0], popt[1], popt[2])
df_merged[f"exp_fit"] = y_fit

In [20]:
plt.figure(figsize=(16, 8))
ax = sns.lineplot(data=df_merged, linewidth=1)
ax.set_title(get_title(), fontdict={"fontsize": fontsize})

# ax.set_yscale("log")
for i, line in enumerate(ax.lines):
    if i >= 0:
        line_style = "solid"
        line.set_color(next(color_cycle)[0])
    else:
        line_style = "dashed"
        line.set_color("goldenrod")
    line.set_linestyle(line_style)

ax.tick_params(axis="x", labelsize=fontsize)
ax.tick_params(axis="y", labelsize=fontsize)
ax.xaxis.label.set_size(fontsize)
ax.yaxis.label.set_size(fontsize)

ax.set_xlabel("Steps")
ax.set_ylabel("Count")

# labels = ["Random Fault"]
labels = []
labels.extend([get_label(simulation_type, n) for n in selected_nodes])
labels.append(
    f"$y = {popt[0]:.2f}e^{{{-popt[1]:.2f}x}} {'+' if popt[2]>=0 else ''} {popt[2]:.2f}$"
)

custom_lines = [
    mlines.Line2D(
        [],
        [],
        color=line.get_color(),
        # marker=marker,
        label=cat,
        linestyle=line.get_linestyle(),
    )
    for line, cat in zip(ax.lines, labels)
]
plt.rc("font", size=fontsize)
plt.legend(handles=custom_lines, fontsize=fontsize * 0.9)
file_path = os.path.join(
    plots_dir,
    program,
    f"{get_filename()}.png",
)


plt.savefig(
    file_path,
    bbox_inches="tight",
)
plt.close()

print(f"Saved plot(s) for {file_path}")

Saved plot(s) for plots/graph_coloring/graph_coloring__graph_10__0__100000__2__9__controlled_at_node_amit_v2__100.png


In [21]:
df_merged

,Count,exp_fit
Steps,,
0.0,0,18128.971753
1.0,41413,15600.961322
2.0,2406,13375.613957
3.0,25493,11416.693681
4.0,3353,9692.302825
5.0,13761,8174.362630
6.0,2055,6838.156034
7.0,5879,5661.925197
8.0,1184,4626.517210


In [22]:
# Save to file
cols = [
    get_label(simulation_type, n) for n in selected_nodes
]
cols.append("exp_curve")
# df_merged.index = df_merged.index.astype(int)
df_merged.to_csv(os.path.join("results", program, f"w_exp_{get_filename()}.csv"))